# Project 5
# Submitted By: Prateek Kumar
### Notebook 1

In [1]:
import pandas as pd #importing the pandas library

In [2]:
user_profiles = pd.read_table('userid-profile.tsv', sep = '\t') #reading the user profiles data

In [3]:
#setting the column values of user_profiles data
user_profiles.columns = ['user_id','gender','age','country','r_date'] 

In [13]:
#the song data contains 19M rows which will take a lot of time to run so 
#we just subset the data hence taking the first 500000 records 
user_data = pd.read_table('userid-timestamp-artid-artname-traid-traname.tsv', error_bad_lines=False, 
                          header = None, sep = '\t', nrows = 500000,
                         names = ['user_id','timestamp','artist_id','artist_name','track_id','track_name'])

In [14]:
user_profiles.head() #user profiles data

,user_id,gender,age,country,r_date
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"


In [15]:
user_data.head() #song data

,user_id,timestamp,artist_id,artist_name,track_id,track_name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [16]:
#checking the dimensions of both of our dataframes
print "User Profiles dimensions:", user_profiles.shape
print "User Profiles dimensions:", user_data.shape

User Profiles dimensions: (992, 5)
User Profiles dimensions: (500000, 6)


In [17]:
# We now merge both the dataframes to one using inner join
result = user_data.merge(user_profiles,how='inner', left_on='user_id', right_on='user_id')

In [18]:
result.shape #the dimension of our merged dataset

(500000, 10)

In [19]:
#we still have a lot of data which contains a lot of missing values 

#The right procedure should be to treat the missing values and draw inferences from it but
#I here dropped the rows with NaN values inorder to draw inferences on the data we have
result = result.dropna() 
result.shape

(276774, 10)

# 1. Recovering the song statistics.

## The number of times a song has been played

In [20]:
print "Unique tracks in our data", len(set(result['track_name'])) #unique tracks in our data
print "Unique users in our data", len(set(result['user_id'])) ##unique users in our data

Unique tracks in our data 43887
Unique users in our data 12


In [21]:
newDF = pd.DataFrame(columns=['track_name','count']) #An empty dataframe to store the values

for i in set(result['track_name']):
    newDF.loc[len(newDF)] = [i, len(result[result['track_name']==i])]

In [22]:
#Displaying the dataframe in descending orer of times the song has been played
newDF.sort_values(by=['count'], ascending=False).head() 

,track_name,count
40689,Heartless,2120
40015,Say You Will,2069
33108,See You In My Nightmares,2069
20584,Love Lockdown,2060
1295,Welcome To Heartbreak (Feat. Kid Cudi),2059


In [66]:
newDF.describe(include='all') #describing all the columns of the dataframe

,track_name,count
count,43887,43887
unique,43887,159
top,Space Suit,1
freq,1,16620


Here we got 43887 rows in the dataframe. The top value of track_name is "Space Suit".

## Percentage of the plays that came from users who self-identified as male

In [23]:
#filtering the male data from our merged data
df_temp = result[result['gender']=='m']

newDF1 = pd.DataFrame(columns=['track_name','count_m']) #creating an empty dataframe to store the values

for i in set(df_temp['track_name']):
    newDF1.loc[len(newDF1)] = [i, len(df_temp[df_temp['track_name']==i])]

#newDF1 = newDF1.sort_values(by=['count_m'], ascending=False)

lis1=[]

#calculating the percentage and storing it to the list and then concatenating it to the dataframe
for j in newDF1['count_m']:
    lis1.append(round((j/len(df_temp['track_name']))*100,2))

newDF1 = pd.concat([newDF1,pd.DataFrame(lis1, columns = ["percentage"])],axis=1) #concat

In [24]:
del newDF1['count_m'] #deleting the less useful values

In [25]:
#Displaying the dataframe in descending orer of male percentage
newDF1.sort_values(by=['percentage'], ascending=False).head()

,track_name,percentage
0,Mr. Misery,0.0
19629,What Can I Do,0.0
19627,Long Dark Night,0.0
19626,Radio Clash,0.0
19625,Nothing From Nothing,0.0


In [67]:
newDF1.describe(include='all') #describing all the columns of the dataframe

,track_name,percentage
count,29430,29430.0
unique,29430,NaN
top,Baby Your Baby,NaN
freq,1,NaN
mean,NaN,0.0
std,NaN,0.0
min,NaN,0.0
25%,NaN,0.0
50%,NaN,0.0
75%,NaN,0.0


Here we got 29430 rows in the dataframe. The top value of track_name is "Baby Your Baby".

## Average age of the listener

In [27]:
newDF2 = pd.DataFrame(columns=['track_name','average_age']) #creating an empty dataframe to store the values

#calculating the average age of the listener
for i in set(result['track_name']):
    newDF2.loc[len(newDF2)] = [i, sum(result[result['track_name']==i]['age'].astype(int))/len(result[result['track_name']==i])]

In [28]:
newDF2.head()

,track_name,average_age
0,Mr. Misery,27
1,Alice In My Chamber (Dawn Of Ashes Remix),19
2,Cissy Strut,28
3,Violet Hill,19
4,Lights In The Sky,25


In [52]:
newDF2['average_age'].unique() #the unique values of average age of the listeners

array([27, 19, 28, 25, 22, 21, 38, 23, 29, 26, 24, 32, 20, 30, 34, 33, 37,
       31, 35, 36], dtype=object)

In [68]:
newDF2.describe(include='all') #describing all the columns of the dataframe

,track_name,average_age
count,43887,43887
unique,43887,20
top,Space Suit,22
freq,1,11252


Here we got 43887 rows in the dataframe. The top value of track_name is "Space Suit".

## Percentage of plays that came from a user who played the song exactly once

In [29]:
#selecting track_name and user_id from our merged data
df_temp2 = result.groupby(['track_name', 'user_id']).size()
df_temp2 = df_temp2.reset_index(level=[0,1])
df_temp2.columns=['track_name','user_id','count']

newDF3 = df_temp2[df_temp2['count'] == 1] #finding the count for songs played exactly once

#Empty dataframe to store the values
df_temp3 = pd.DataFrame(columns=['track_name','count_once','total_count','percentage'])

#Assigning the values to the dataframe
for i in set(result['track_name']):
    count_once = sum(newDF3['track_name']==i)
    count_total = sum(result['track_name']==i)
    df_temp3.loc[len(df_temp3)] = [i, count_once, count_total, (count_once/count_total)*100]

In [30]:
del df_temp3['count_once'] #removing the less important values

In [31]:
del df_temp3['total_count'] #removing the less important values

In [32]:
df_temp3.head()

,track_name,percentage
0,Mr. Misery,0
1,Alice In My Chamber (Dawn Of Ashes Remix),100
2,Cissy Strut,0
3,Violet Hill,0
4,Lights In The Sky,0


In [69]:
df_temp3.describe(include='all') #describing all the columns of the dataframe

,track_name,percentage
count,43887,43887
unique,43887,2
top,Space Suit,0
freq,1,26562


Here we got 43887 rows in the dataframe. The top value of track_name is "Space Suit".

## Percentage of plays that came from a user who played the song at least five times

In [34]:
newDF4 = df_temp2[df_temp2['count'] > 4] #filtering the data for at least five 

#Empty dataframe to store the values
df_temp4 = pd.DataFrame(columns=['track_name','count_more','total_count','percentage'])

#Assigning the values to the dataframe
for i in set(result['track_name']):
    count_more = sum(newDF4[newDF4['track_name']==i]['count'])
    count_total = sum(result['track_name']==i)
    df_temp4.loc[len(df_temp4)] = [i, count_more, count_total, (count_more/count_total)*100]

In [35]:
del df_temp4['count_more'] #removing the less important values

In [36]:
del df_temp4['total_count'] #removing the less important values

In [37]:
df_temp4.head()

,track_name,percentage
0,Mr. Misery,0
1,Alice In My Chamber (Dawn Of Ashes Remix),0
2,Cissy Strut,100
3,Violet Hill,0
4,Lights In The Sky,0


In [70]:
df_temp4.describe(include='all') #describing all the columns of the dataframe

,track_name,percentage
count,43887,43887
unique,43887,2
top,Space Suit,0
freq,1,33560


Here we got 43887 rows in the dataframe. The top value of track_name is "Space Suit".

## Joining all the five dataframes to one

In [39]:
result_new = newDF.merge(newDF1,how='inner', left_on='track_name', right_on='track_name')

In [40]:
result_new = result_new.merge(newDF2,how='inner', left_on='track_name', right_on='track_name')

In [41]:
result_new = result_new.merge(df_temp3,how='inner', left_on='track_name', right_on='track_name')

In [42]:
result_new = result_new.merge(df_temp4,how='inner', left_on='track_name', right_on='track_name')

In [43]:
result_new.columns = ['track_name', 'play_times', 'percent_male', 'average_age_listener', 'percent_once', 'percent_five+']

In [63]:
result_new.tail()

,track_name,play_times,percent_male,average_age_listener,percent_once,percent_five+
29425,Banquet (Phones Disco Edit),1,0.0,22,100,0
29426,Lust,10,0.0,25,0,0
29427,The Devil Game,2,0.0,22,0,0
29428,Flight Of The Phoenicians,17,0.0,27,0,100
29429,Wake Up Screaming,1,0.0,22,100,0


In [45]:
result_new.shape #dimensions of the final dataframe

(29430, 6)

# 2. Descriptive Statistics

In [57]:
result_new.describe(include='all') #describing all the columns of the new dataframe

,track_name,play_times,percent_male,average_age_listener,percent_once,percent_five+
count,29430,29430.0,29430.0,29430.0,29430.0,29430.0
unique,29430,145.0,NaN,20.0,2.0,2.0
top,Baby Your Baby,1.0,NaN,22.0,0.0,0.0
freq,1,11143.0,NaN,11225.0,17676.0,23234.0
mean,NaN,NaN,0.0,NaN,NaN,NaN
std,NaN,NaN,0.0,NaN,NaN,NaN
min,NaN,NaN,0.0,NaN,NaN,NaN
25%,NaN,NaN,0.0,NaN,NaN,NaN
50%,NaN,NaN,0.0,NaN,NaN,NaN
75%,NaN,NaN,0.0,NaN,NaN,NaN


As we can see in the above description of the final generated datframe .

The describe function generates descriptive statistics that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values. It analyzes both numeric and object series, as well as DataFrame column sets of mixed data types. 

As asked process this data to recover some song statistics I filtered the datas based upon the each track_name i.e. average age of the listener is calculated based upon the average age of the listener per track_name.

Description of the columns:

1. track_name: 
(i) The count values shows the length of the column i.e. the number of rows, so it will be 29430 for all columns
(ii) All the data in our final dataframe is unique so the unique value is same as the count value
(iii) The top song is "Baby Your Baby"

2. play_times:
(i) We have 145 different song play count values

3. percent_male:
(i) We are not getting any relevant values here

4. average_age_listener:
(i) We have 20 unique ages in the dataframe
(ii) The top age is 22

5. percent_once:
(i) We have just 2 unique values 0 and 100 because of less data.

6. percent_five+:
(i) Here as well we have just 2 unique values 0 and 100 because of less data.

# 3. Pickle

In [46]:
import pickle  # importing the pickle library

In [47]:
# creating the pickle file of the final dataframe

pickle_out = open('data.pkl', 'wb')

# Pickle the 'data' dictionary using the highest protocol available.
pickle.dump(result_new, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()

In [48]:
# creating the pickle file of the initial master dataframe
pickle_out = open('data1.pkl', 'wb')

# Pickle the 'data' dictionary using the highest protocol available.
pickle.dump(result, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()